<a href="https://colab.research.google.com/github/Kat-nesh/Redefining-maize-UAI/blob/main/PrepareImg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from geemap import ml
from sklearn import ensemble
from geemap import ml
from sklearn import ensemble
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
plt.rcParams.update({'figure.figsize': (12.0,8.0)})
plt.rcParams.update({'font.size' : 14})

In [ ]:
#test img
from IPython.core.magics.execution import Restart
try:
  import geemap
  import ee
  import geopandas
  import math
except Exception as e:
  !pip install geopandas
  !pip install geemap
  Restart()
finally:
  import geemap
  import ee
try:
  ee.Initialize(project='ee-maizeyield')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-maizeyield')


In [ ]:
user_id = geemap.ee_user_id()
user_id

'projects/ee-maizeyield/assets/LR_18'

# Add Indices

In [ ]:
# ADD INDICES
def addIndices(image,AOI):

  rangeBands = {
                      'blue': image.select('blue'),
                      'green':image.select('green'),
                      'red': image.select('red'),
                      'nir': image.select('nir'),
                      'swir1': image.select('swir1'),
                      'swir2': image.select('swir2')
                    }
   #NDVI
  ndvi = image.expression('(nir - red) / (nir + red)',rangeBands)

  # SAVI
  savi = image.expression('1.5 * ((nir - red) / (nir + red + 0.5))',rangeBands)

  #EVI
  evi = image.expression('2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1))',rangeBands)

  #Green Normalized difference Vegetation Index (GNDVI)

  gndvi = image.expression('(nir - green) / (nir + green)',rangeBands)

  #Normalize Differennce Mositure Index (NDMI)
  ndmi = image.expression('(nir - swir1) / (nir + swir1)',rangeBands)

  # normalized special index (NSI)
  nsi = image.expression('100 * ((swir1 - nir) / (nir - red))',rangeBands)

  #Modified Soil Adjusted Vegetation Index MSAVI
  msavi =image.expression('(2 * nir + 1 - sqrt(pow((2 * nir + 1), 2) - 8 * (nir - red)) ) / 2',rangeBands)

  #Green Chlorophyll Index (GCI)
  gci =image.expression('(nir/green) - 1',rangeBands)

  #Green Leaf Index (GLI)
  gli =image.expression('((green - red) + (green - blue)) / ((2 * green) + red + blue)',rangeBands)

  # Structure Insensitive Pigment Index (sipi)
  sipi =image.expression('(nir - blue) / (nir - red)',rangeBands)

  # Simple Ration RVI (Ratio Vegetation Index)
  sr =image.expression('nir/red',rangeBands)

  # OSAVI (Optimized Soil-Adjusted Vegetation Index):
  # ((NIR - Red) / (NIR + Red + 0.16)) * (1 + 0.16)
  osavi =image.expression('((nir - red) / (nir + red + 0.16)) * (1 + 0.16)',rangeBands)

  # GCI2 (Green Chlorophyll Index 2):
  # (NIR - 2.4 * Red + Blue) / (NIR + 2.4 * Red + Blue)
  gci2 =image.expression('(nir - 2.4 * red + blue) / (nir + 2.4 * red + blue)',rangeBands)

  # ARVI2 (Atmospherically Resistant Vegetation Index 2):
  # (nir - (2 * red - blue)) / (nir + (2 * red - blue))
  arvi2 =image.expression('(nir - (2 * red - blue)) / (nir + (2 * red - blue))',rangeBands)

  # MTVI2 (Modified Triangular Vegetation Index 2):
  # 1.5 * (1.2 * (nir - green) - 2.5 * (red - green)) / sqrt((2 * nir + 1)^2 - (6 * nir - 5 * sqrt(red) - 0.5))
  mtvi2 =image.expression('1.5 * (1.2 * (nir - green) - 2.5 * (red - green)) / sqrt((2 * nir + 1)**2 - (6 * nir - 5 * sqrt(red) - 0.5))',rangeBands)

  # RDVI (Renormalized Difference Vegetation Index):
  # (nir - red) / sqrt(nir + red)
  rdvi =image.expression('(nir - red) / sqrt(nir + red)',rangeBands)

  # GVI (Global Vegetation Index):
  # (nir / (nir + green)) - (red / (red + blue))
  gvi =image.expression('(nir / (nir + green)) - (red / (red + blue))',rangeBands)


  # NDII (Normalized Difference Infrared Index):
  # (nir - swir2) / (nir + swir2)
  ndii =image.expression('(nir - swir2) / (nir + swir2)',rangeBands)

  # where swir2 is the shortwave-infrared band centered at 2215 nm.


  # NDWI (Normalized Difference Water Index): # (green - nir) / (green + nir)
  ndwi =image.expression('(green - nir) / (green + nir)',rangeBands)

  # GRVI (Green-Red Vegetation Index): # Green / Red
  grvi =image.expression('green / red',rangeBands)

  # LAI (Leaf Area Index): # 3.618 * ((nir - red) / (nir + red)) - 0.118
  lai =image.expression('3.618 * ((nir - red) / (nir + red)) - 0.118',rangeBands)

  # TCARI (Transformed Chlorophyll Absorption in Reflectance Index):
  # 3 * ((nir - red) - 0.2 * (red - green) * (nir / red))
  tcari =image.expression('3 * ((nir - red) - 0.2 * (red - green) * (nir / red))',rangeBands)

  # MCARI (Modified Chlorophyll Absorption in Reflectance Index):
  # (1.5 * (2.5 * (nir - green) - 1.3 * (nir - red))) / sqrt((2 * nir + 1)^2 - (6 * nir - 5 * sqrt(red) - 0.5))
  mcari =image.expression(' (1.5 * (2.5 * (nir - green) - 1.3 * (nir - red))) / sqrt((2 * nir + 1)**2 - (6 * nir - 5 * sqrt(red) - 0.5))',rangeBands)

  # WDVI (Weighted Difference Vegetation Index):
  # nir - 0.5 * red
  wdvi =image.expression('nir - 0.5 * red',rangeBands)

  # PSSRa (Plant Senescence Reflectance Ratio Index):
  # (red / nir) / (red / green)
  pssra =image.expression('(red / nir) / (red / green)',rangeBands)

  # DVI (Difference Vegetation Index):
  # nir - red
  dvi =image.expression('nir - red',rangeBands)


  image = image.addBands(ndvi.clip(AOI).rename('ndvi'))\
                .addBands(savi.clip(AOI).rename('savi'))\
                .addBands(evi.clip(AOI).rename('evi'))\
                .addBands(gndvi.clip(AOI).rename('gndvi'))\
                .addBands(ndmi.clip(AOI).rename('ndmi'))\
                .addBands(nsi.clip(AOI).rename('nsi'))\
                .addBands(msavi.clip(AOI).rename('msavi'))\
                .addBands(gci.clip(AOI).rename('gci'))\
                .addBands(gli.clip(AOI).rename('gli'))\
                .addBands(sipi.clip(AOI).rename('sipi'))\
                .addBands(sr.clip(AOI).rename('sr'))\
                .addBands(osavi.clip(AOI).rename('osavi'))\
                .addBands(gci2.clip(AOI).rename('gci2'))\
                .addBands(arvi2.clip(AOI).rename('arvi2'))\
                .addBands(mtvi2.clip(AOI).rename('mtvi2'))\
                .addBands(rdvi.clip(AOI).rename('rdvi'))\
                .addBands(gvi.clip(AOI).rename('gvi'))\
                .addBands(ndii.clip(AOI).rename('ndii'))\
                .addBands(ndwi.clip(AOI).rename('ndwi'))\
                .addBands(grvi.clip(AOI).rename('grvi'))\
                .addBands(lai.clip(AOI).rename('lai'))\
                .addBands(tcari.clip(AOI).rename('tcari'))\
                .addBands(mcari.clip(AOI).rename('mcari'))\
                .addBands(wdvi.clip(AOI).rename('wdvi'))\
                .addBands(pssra.clip(AOI).rename('pssra'))\
                .addBands(dvi.clip(AOI).rename('dvi'))

  return image.float()

# Prepare L8

In [ ]:
# Prepare L8
def l8prep(start_date,end_date,image_col,AOI):
  # Load the L8 Collection and Processs
  temp_date = ee.Date("2013-02-01")
  if start_date.millis().lt(temp_date.millis()).getInfo():
    l8 = image_col.filterDate(start_date,end_date).filterBounds(AOI).select('SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B7')
  else:
    l8 = image_col.filterDate(start_date,end_date).filterBounds(AOI).select('SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6','SR_B7')

  l8_mean = l8.mean().rename(['blue','green','red','nir','swir1','swir2'])
  l8_median = l8.median().rename(['blue','green','red','nir','swir1','swir2'])
  l8_min = l8.min().rename(['blue','green','red','nir','swir1','swir2'])
  l8_max = l8.max().rename(['blue','green','red','nir','swir1','swir2'])
  l8_sd = l8.reduce(ee.Reducer.stdDev()).rename(['blue','green','red','nir','swir1','swir2'])
  l8_pc10 = l8.reduce(ee.Reducer.percentile([10])).rename(['blue','green','red','nir','swir1','swir2'])
  l8_pc50 = l8.reduce(ee.Reducer.percentile([50])).rename(['blue','green','red','nir','swir1','swir2'])
  l8_pc90 = l8.reduce(ee.Reducer.percentile([90])).rename(['blue','green','red','nir','swir1','swir2'])
  return {
        'mean': l8_mean,
        'median': l8_median,
        'min': l8_min,
        'max': l8_max,
        'stdDev': l8_sd,
        'percentile10': l8_pc10,
        'percentile50': l8_pc50,
        'percentile90': l8_pc90
    }

In [ ]:
start = ee.Date("2010-04-01")
end = ee.Date("2023-08-31")
prod = ee.FeatureCollection("projects/ee-maizeyield/assets/county_maize_yield_2010-18")
AOI = prod.geometry()
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")#.filterDate(start,end).filterBounds(AOI).select('SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6','SR_B7')
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")#.filterDate(start,end).filterBounds(AOI).select('SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B7')

In [ ]:
Map= geemap.Map()
Map.centerObject(AOI,6)
Map.addLayer(AOI)

In [ ]:
def processL8 (start, end, img, AOI,type):
  l8_results =  l8prep(start, end, img, AOI)
  mean = l8_results [type]
  idx = addIndices(mean,AOI)
  return idx

# Processed L8

In [ ]:
scale = 463.3127165275

In [ ]:
year = '2010'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2010 =processL8(start,end,l7,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2010 =processL8(start,end,l7,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2010 =processL8(start,end,l7,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2010 =processL8(start,end,l7,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2010 =processL8(start,end,l7,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2010 =processL8(start,end,l7,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2010 =processL8(start,end,l7,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2010 =processL8(start,end,l7,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})


l8_2010_prj = l8_2010.reproject('EPSG:4326',scale=scale)
# l8_2010_prj_red = l8_2010_prj.reduceResolution(reducer=ee.Reducer.mean())

# export_params = {
#   'assetId': user_id +'/Outputs/Out5k/l8_2010',
#   'region': AOI,
#   'scale': 30,
#   'maxPixels': 1e13
# }

# # Export the image to the asset
# task = ee.batch.Export.image.toAsset(l8_2010, description='l8_2010', **export_params)
# task.start()

In [ ]:
#l8_2010_prj.getInfo()#['bands'][0]['crs']
# Map.addLayer(l8_2010_prj,{'min':0,'max':10000},'l8_2010')
# Map.addLayer(l8_2010_prj_red,{'min':0,'max':10000},'l8_2010_red')
# Map

In [ ]:
year = '2011'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2011 =processL8(start,end,l7,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2011 =processL8(start,end,l7,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2011 =processL8(start,end,l7,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2011 =processL8(start,end,l7,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2011 =processL8(start,end,l7,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2011 =processL8(start,end,l7,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2011 =processL8(start,end,l7,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2011 =processL8(start,end,l7,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})
l8_2011_prj = l8_2011.reproject('EPSG:4326',scale=scale)
# export_params = {
#   'assetId': user_id +'/Outputs/Out5k/l8_2011',
#   'region': AOI,
#   'scale': 30,
#   'maxPixels': 1e13
# }

# # Export the image to the asset
# task = ee.batch.Export.image.toAsset(l8_2011, description='l8_2011', **export_params)
# task.start()

In [ ]:
year = '2012'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2012 =processL8(start,end,l7,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2012 =processL8(start,end,l7,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2012 =processL8(start,end,l7,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2012 =processL8(start,end,l7,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2012 =processL8(start,end,l7,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2012 =processL8(start,end,l7,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2012 =processL8(start,end,l7,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2012 =processL8(start,end,l7,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2012_prj = l8_2012.reproject('EPSG:4326',scale=scale)

# export_params = {
#   'assetId': user_id +'/Outputs/Out5k/l8_2012',
#   'region': AOI,
#   'scale': 30,
#   'maxPixels': 1e13
# }

# # Export the image to the asset
# task = ee.batch.Export.image.toAsset(l8_2012, description='l8_2012', **export_params)
# # task.start()

In [ ]:
# task.status()

In [ ]:
year = '2013'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2013 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2013 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2013 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2013 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2013 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2013 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2013 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2013 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2013_prj = l8_2013.reproject('EPSG:4326',scale=scale)

In [ ]:
year = '2014'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2014 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2014 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2014 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2014 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2014 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2014 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2014 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2014 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2014_prj = l8_2014.reproject('EPSG:4326',scale=scale)


In [ ]:
year = '2015'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2015 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2015 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2015 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2015 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2015 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2015 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2015 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2015 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2015_prj = l8_2015.reproject('EPSG:4326',scale=scale)


In [ ]:
year = '2016'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2016 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2016 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2016 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2016 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2016 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2016 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2016 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2016 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2016_prj = l8_2016.reproject('EPSG:4326',scale=scale)



In [ ]:
year = '2017'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2017 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2017 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2017 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2017 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2017 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2017 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2017 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2017 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2017_prj = l8_2017.reproject('EPSG:4326',scale=scale)


In [ ]:
year = '2018'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2018 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2018 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2018 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2018 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2018 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2018 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2018 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2018 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2018_prj = l8_2018.reproject('EPSG:4326',scale=scale)


In [ ]:
year = '2019'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2019 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2019 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2019 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2019 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2019 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2019 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2019 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2019 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2019_prj = l8_2019.reproject('EPSG:4326',scale=scale)


In [ ]:
year = '2020'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2020 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2020 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2020 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2020 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2020 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2020 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2020 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2020 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2020_prj = l8_2020.reproject('EPSG:4326',scale=scale)


In [ ]:
year = '2021'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2021 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2021 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2021 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2021 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2021 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2021 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2021 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2021 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2021_prj = l8_2021.reproject('EPSG:4326',scale=scale)


In [ ]:
year = '2022'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2022 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2022 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2022 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2022 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2022 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2022 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2022 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2022 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2022_prj = l8_2022.reproject('EPSG:4326',scale=scale)


In [ ]:
year = '2023'
start = ee.Date(year+'-04-01')
end   = ee.Date(year+'-08-31')
l8_2023 =processL8(start,end,l8,AOI,'mean').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2023 =processL8(start,end,l8,AOI,'median').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2023 =processL8(start,end,l8,AOI,'min').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2023 =processL8(start,end,l8,AOI,'max').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2023 =processL8(start,end,l8,AOI,'stdDev').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2023 =processL8(start,end,l8,AOI,'percentile10').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2023 =processL8(start,end,l8,AOI,'percentile50').set({"Year":year,'Name':year+'_L8_processed'})
# l8_2023 =processL8(start,end,l8,AOI,'percentile90').set({"Year":year,'Name':year+'_L8_processed'})

l8_2023_prj = l8_2023.reproject('EPSG:4326',scale=scale)


In [ ]:
# imageList = [l8_2010,l8_2011,l8_2012,l8_2013,l8_2014,l8_2015,l8_2015,l8_2016,l8_2017,l8_2018,
#              l8_2019,l8_2020,l8_2021,l8_2022,l8_2023]
imageList = [l8_2010_prj,l8_2011_prj,l8_2012_prj,l8_2013_prj,l8_2014_prj,l8_2015_prj,l8_2016_prj,
             l8_2017_prj,l8_2018_prj,l8_2019_prj,l8_2020_prj,l8_2021_prj,l8_2022_prj,l8_2023_prj]

In [ ]:
# // Merge the list of images into an image collection
imgcol = ee.ImageCollection.fromImages(imageList)

In [ ]:
imgcol_ = imgcol.toBands()
# imgcol_


# Export L8 CSV

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2010.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2010.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2011.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2011.csv')

In [ ]:
# Reduce Regions to Counties
# test_reg = l8_2012.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2012.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2013.reduceRegions(**{
#     'collection':prod,
# #     'reducer': ee.Reducer.mean(),
# #     'scale': 30,

# })

In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2013.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2014.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2014.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2015.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2015.csv')

In [ ]:
# Reduce Regions to Counties
# test_reg = l8_2016.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2016.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2017.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2017.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2018.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2018.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2019.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2019.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2020.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,
# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2020.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2021.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2021.csv')

In [ ]:
# Reduce Regions to Counties
# test_reg = l8_2022.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2022.csv')

In [ ]:
# # Reduce Regions to Counties
# test_reg = l8_2023.reduceRegions(**{
#     'collection':prod,
#     'reducer': ee.Reducer.mean(),
#     'scale': 30,

# })


In [ ]:
# geemap.ee_export_vector(test_reg, 'l8_agg_2023.csv')

# Process MODIS

In [ ]:

def getQABits(image, start, end, newName):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start, end, 1):
      pattern += math.pow(2, i)

    # Return a single band image of the extracted QA bits, giving the band
    # a new name.
    return image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)


In [ ]:

# ---Creat Masks based on StateQA (band12) and QA (band8) bands FOR mod09
def maskStateQA(image):
  # Select the StateQA band.
  QA = image.select('StateQA')
  #Get cloud and shadow flag bit.
  cloud = getQABits(QA, 0, 2, 'cloud_shadow') \
                        .expression("b(0)==0")
  #Get water flag bit.
  water = getQABits(QA, 3, 5, 'water_state') \
                        .expression("b(0)==1")
  #Get aerosol flags (https:#earthscience.stackexchange.com/questions/10046/modis-surface-reflectance-data-state-qa-aerosol-quantity-field-meaning-is-unclea)
  #https:#www.mdpi.com/2072-4292/5/7/3495/htm
  aerosol = getQABits(QA, 6, 7, 'aerosol_state') \
                        .expression("b(0)==1")
  #Cirrus detected
  cirrus = getQABits(QA, 8, 9, 'cirrus_state') \
                        .expression("b(0)==0")
  #Fire and snow
  otherstates = getQABits(QA, 11, 12, 'other_states') \
                        .expression("b(0)==0")
  # Select the QA band now.
  QA = image.select('QA')
  #Get band quality state flags bit.NB: ITS VERY STRICT TAKES OU MOST PIXELS
  #bandqa = getQABits(QA, 0, 29, 'band_quality')
                        #.expression("b(0)==0")
  #atmospheric & adjancency correction performed state flags bit.
  corrected = getQABits(QA, 30, 31, 'state_corr')
                        #.expression("b(0)==1")
  # Return an image masking specified areas.
  image = image.updateMask(cloud)
  image = image.updateMask(water)
  image = image.updateMask(aerosol)
  image = image.updateMask(cirrus)
  image = image.updateMask(otherstates)
  #image = image.updateMask(bandqa)
  image = image.updateMask(corrected)
  return image

In [ ]:

# ---Create FPAR and LAI Masks based on StateQA (band12) and QA (band8) bands FOR mod09
def MOD15A2Hmask(image):
  # Select the StateQA band.
  QA = image.select('FparExtra_QC')
  #Get cloud and shadow flag bit.
  cloud = getQABits(QA, 0, 6, 'No_atmophericEffects') \
                        .expression("b(0)==0")
  # Return an image masking specified areas.
  image = image.updateMask(cloud)
  return image


In [ ]:
# ---Creat GPP Masks based on StateQA (band12) and QA (band8) bands FOR mod09
def MOD17mask(image):
  # Select the StateQA band.
  QA = image.select('Psn_QC')
  #Get good quality flag bit.
  qc = getQABits(QA, 0, 7, 'good_quality').expression("b(0)==0")
  # Return an image masking specified areas.
  image = image.updateMask(qc)
  return image

In [ ]:
# #  * Function to mask clouds using the Sentinel-2 QA band

def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

In [ ]:
# Prepare GPP
def gpp_prep(image_col,AOI,year):
  img = image_col.filterDate(year+'-01-01',year+'-12-31').filterBounds(AOI)
  #Import NPP -----Annual
  gpp = img.select('Gpp')
  return gpp


In [ ]:
# npp_col = ee.ImageCollection('MODIS/061/MOD17A3HGF')#.select('Npp').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# gpp_col = ee.ImageCollection("MODIS/006/MOD17A2H")#.select('Gpp')

In [ ]:
# img = gpp_col.filterDate(start,end).filter(ee.Filter.calendarRange(4, 8,'month')).filterBounds(AOI)
# msk = img.mean()
# msk
# QA = msk.select('Psn_QC')
# # print(QA.getInfo())
# #   #Get good quality flag bit.
# qc = getQABits(QA, 0, 7, 'good_quality').expression("b(0)==0")
# # qc.getInfo()
# #   # Return an image masking specified areas.
# # image = image.updateMask(qc)

In [ ]:
# Prepare NPP
def npp_prep(image_col,AOI,year):
  img = image_col.filterDate(year+'-01-01',year+'-12-31').filterBounds(AOI)
  #Import NPP -----Annual
  npp = img.select('Npp')
  return npp


In [ ]:
# Prepare FPAR
def fpar_prep(start_date,end_date,image_col,AOI):
  img = image_col.filterDate(start_date,end_date).filter(ee.Filter.calendarRange(4, 8,'month')).filterBounds(AOI)
  # Import Fraction of Photosynthetically Active Radiation (FPAR)
  fpar = MOD15A2Hmask(img.mean()).select('Fpar_500m')
  return fpar


In [ ]:
# Prepare LAI
def lai_prep(start_date,end_date,image_col,AOI):
  img = image_col.filterDate(start_date,end_date).filter(ee.Filter.calendarRange(4, 8,'month')).filterBounds(AOI)
  # Import Leaf Area Index
  lai  = MOD15A2Hmask(img.mean()).select('Lai_500m')
  return lai

# Climatic

# Pre Req

In [ ]:
start= ee.Date('2010-04-01')
end = ee.Date('2023-08-31')
bound = ee.FeatureCollection("users/Prod/boundary")
newzones = ee.FeatureCollection("users/Prod/zone_5")
AOI = bound.geometry()
Map = geemap.Map()
Map.centerObject(AOI, 6)
Map.addLayer(AOI,{},'AOI')

In [ ]:
county = ee.FeatureCollection("projects/ee-maizeyield/assets/county_maize_yield_2010-18")
scale = 463.31271652791656
years = ee.List.sequence(2010,2023)
# years.getInfo()

# Visualization

In [ ]:
# // Visualization palette for total precipitation
visTp = {
  'min': 0,
  'max': 0.1,
  'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

In [ ]:
# // Visualization palette for Temperature
vis2mt = {
  'min': 250,
  'max': 320,
  'palette': [
    '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff', '00ff80',
    '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00', 'ffb000', 'ffa400',
    'ff4f00', 'ff2500', 'ff0a00', 'ff00ff'
  ]
}

In [ ]:
# // Visualization palette for u- and v-component of 10m wind
visWind = {
  'min': 0,
  'max': 30,
  'palette': [
    '#FFFFFF', '#FFFF71', '#DEFF00', '#9EFF00', '#77B038', '#007E55', '#005F51',
    '#004B51', '#013A7B', '#023AAD'
  ]
}

In [ ]:
visMtp = {
  'min': 250.0,
  'max': 320.0,
  'palette': [
    "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
    "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
    "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
  ]
}

In [ ]:
visChirps = {
  'min': 1.0,
  'max': 17.0,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
};

# ERA 5 Rainfall

In [ ]:

# // Daily total precipitation sums
era5_tp = ee.ImageCollection('ECMWF/ERA5/DAILY').select('total_precipitation').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month')).filterBounds(AOI)
era_md = era5_tp.median().clip(AOI)
# Map.addLayer(era_md,visTp,'ERA5 Total Prec')

In [ ]:
# */
# //--------------- Get the MEAN--------------------------------------
era5_tp_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5_tp.filter(ee.Filter.calendarRange(year, year, 'year')).mean().set('Year',year)))
# .rename(['blue','green','red','nir','swir1'])))
# era5_tp_mean


In [ ]:
# era5_tp_mean.toBands().bandNames().getInfo()

In [ ]:
era5_tp_mean_=era5_tp_mean.toBands().select(['0_total_precipitation', '1_total_precipitation', '2_total_precipitation'
, '3_total_precipitation', '4_total_precipitation','5_total_precipitation', '6_total_precipitation', '7_total_precipitation',
 '8_total_precipitation','9_total_precipitation','10_total_precipitation'
#  ,'11_total_precipitation','12_total_precipitation','13_total_precipitation'
 ],[
'era5d_2010_total_precipitation','era5d_2011_total_precipitation','era5d_2012_total_precipitation','era5d_2013_total_precipitation',
'era5d_2014_total_precipitation','era5d_2015_total_precipitation','era5d_2016_total_precipitation','era5d_2017_total_precipitation',
'era5d_2018_total_precipitation',
 'era5d_2019_total_precipitation','era5d_2020_total_precipitation',
    #  'era5d_2021_total_precipitation' ,'era5d_2022_total_precipitation',  'era5d_2023_total_precipitation'
])
# era5_tp_mean_.getInfo()

In [ ]:
# Map.addLayer(era_md,visTp,'ERA5 Total Prec')

In [ ]:
era5_monthly = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('total_precipitation').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month')).filterBounds(AOI)
# era_mmd = era5_monthly.median().clip(AOI)
# Map.addLayer(era_mmd,visMtp,'ERA5 Montly Prec')
# era5_monthly

In [ ]:
# */
# //--------------- Get the MEAN--------------------------------------
era5_monthly_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5_monthly.filter(ee.Filter.calendarRange(year, year, 'year')).mean()))
# .rename(['blue','green','red','nir','swir1'])))
# era5_monthly_mean

In [ ]:
# era5_monthly_mean.toBands().bandNames().getInfo()

In [ ]:
era5_monthly_mean_=era5_monthly_mean.toBands().select(
  ['0_total_precipitation', '1_total_precipitation', '2_total_precipitation'
  , '3_total_precipitation', '4_total_precipitation','5_total_precipitation', '6_total_precipitation', '7_total_precipitation',
   '8_total_precipitation','9_total_precipitation','10_total_precipitation'
  #  ,'11_total_precipitation','12_total_precipitation','13_total_precipitation'
   ],[
    'era5m_2010_total_precipitation','era5m_2011_total_precipitation','era5m_2012_total_precipitation','era5m_2013_total_precipitation','era5m_2014_total_precipitation'
    ,'era5m_2015_total_precipitation','era5m_2016_total_precipitation','era5m_2017_total_precipitation','era5m_2018_total_precipitation',
    'era5m_2019_total_precipitation','era5m_2020_total_precipitation'
    # ,'era5m_2021_total_precipitation' ,'era5m_2022_total_precipitation','era5m_2023_total_precipitation'
    ])
# era5_monthly_mean_.getInfo()

# ERA 5 Temperature

**Daily Mean Temperature**

In [ ]:
# // Daily Mean Temperature
era5_mean_temp = ee.ImageCollection('ECMWF/ERA5/DAILY').select('mean_2m_air_temperature').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# era_md = era5_tp.median().clip(AOI)
# Map.addLayer(era_md,visTp,'ERA5 Total Temp')
# era5_mean_temp

In [ ]:
# */
# //--------------- Get the MEAN--------------------------------------
era5_mean_temp_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5_mean_temp.filter(ee.Filter.calendarRange(year, year, 'year')).mean().set('Year',year)))
# .rename(['blue','green','red','nir','swir1'])))
# era5_mean_temp_mean

In [ ]:
# era5_mean_temp_mean.toBands().bandNames().getInfo()

In [ ]:
era5_mean_temp_mean_=era5_mean_temp_mean.toBands().select(era5_mean_temp_mean.toBands().bandNames().getInfo()
    # ['0_mean_2m_air_temperature', '1_mean_2m_air_temperature', '2_mean_2m_air_temperature', '3_mean_2m_air_temperature', '4_mean_2m_air_temperature','5_mean_2m_air_temperature', '6_mean_2m_air_temperature',
    #  '7_mean_2m_air_temperature','8_mean_2m_air_temperature','9_mean_2m_air_temperature','10_mean_2m_air_temperature'
    #  ,'11_mean_2m_air_temperature','12_mean_2m_air_temperature','13_mean_2m_air_temperature'
     ,[
 'era5_2010_mean_temp','era5_2011_mean_temp','era5_2012_mean_temp','era5_2013_mean_temp', 'era5_2014_mean_temp',
 'era5_2015_mean_temp', 'era5_2016_mean_temp','era5_2017_mean_temp', 'era5_2018_mean_temp' ,'era5_2019_mean_temp'
 ,'era5_2020_mean_temp'
#  ,'era5_2021_mean_temp','era5_2022_mean_temp' ,'era5_2023_mean_temp'
 ])
# era5_mean_temp_mean_.getInfo()

**Daily Min Temperature**

In [ ]:
# // Daily MIn Temperature
era5_min_temp = ee.ImageCollection('ECMWF/ERA5/DAILY').select('minimum_2m_air_temperature').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# era5_min_temp

In [ ]:
# //--------------- Get the MEAN--------------------------------------
era5_min_temp_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5_min_temp.filter(ee.Filter.calendarRange(year, year, 'year')).mean().set('Year',year)))
# era5_min_temp_mean

In [ ]:
era5_min_temp_mean_=era5_min_temp_mean.toBands().select(era5_min_temp_mean.toBands().bandNames().getInfo()
    # ['0_minimum_2m_air_temperature', '1_minimum_2m_air_temperature', '2_minimum_2m_air_temperature', '3_minimum_2m_air_temperature', '4_minimum_2m_air_temperature','5_minimum_2m_air_temperature', '6_minimum_2m_air_temperature', '7_minimum_2m_air_temperature',
    #  '8_minimum_2m_air_temperature','9_minimum_2m_air_temperature','10_minimum_2m_air_temperature'
    #  ,'11_minimum_2m_air_temperature','12_minimum_2m_air_temperature','13_minimum_2m_air_temperature']
    ,[
 'era5_2010_min_temp','era5_2011_min_temp','era5_2012_min_temp','era5_2013_min_temp', 'era5_2014_min_temp',
 'era5_2015_min_temp','era5_2016_min_temp','era5_2017_min_temp', 'era5_2018_min_temp', 'era5_2019_min_temp','era5_2020_min_temp'
#  ,'era5_2021_min_temp','era5_2022_min_temp' ,'era5_2023_min_temp'
 ])
# era5_min_temp_mean_.getInfo()

**Daily Max Temperature**

In [ ]:
# // Daily Max Temperature
era5_max_temp = ee.ImageCollection('ECMWF/ERA5/DAILY').select('maximum_2m_air_temperature').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# era5_max_temp

In [ ]:
# //--------------- Get the MEAN--------------------------------------
era5_max_temp_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5_max_temp.filter(ee.Filter.calendarRange(year, year, 'year')).mean().set('Year',year)))
# era5_max_temp_mean

In [ ]:
era5_max_temp_mean_=era5_max_temp_mean.toBands().select(era5_max_temp_mean.toBands().bandNames().getInfo()
    #   ['0_maximum_2m_air_temperature', '1_maximum_2m_air_temperature', '2_maximum_2m_air_temperature',
    #    '3_maximum_2m_air_temperature', '4_maximum_2m_air_temperature','5_maximum_2m_air_temperature',
    #    '6_maximum_2m_air_temperature', '7_maximum_2m_air_temperature',
    #  '8_maximum_2m_air_temperature','9_maximum_2m_air_temperature','10_maximum_2m_air_temperature'
    #  ,'11_maximum_2m_air_temperature','12_maximum_2m_air_temperature','13_maximum_2m_air_temperature']
       ,[
 'era5_2010_max_temp','era5_2011_max_temp','era5_2012_max_temp','era5_2013_max_temp', 'era5_2014_max_temp',
 'era5_2015_max_temp','era5_2016_max_temp','era5_2017_max_temp', 'era5_2018_max_temp',
 'era5_2019_max_temp','era5_2020_max_temp'
#  ,'era5_2021_max_temp' ,'era5_2022_max_temp' ,'era5_2023_max_temp'
 ])
# era5_max_temp_mean_.getInfo()

**Monthly Max Temperature**


In [ ]:
era5m_temp = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('mean_2m_air_temperature').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# era_mmd = era5_monthly.median().clip(AOI)
# Map.addLayer(era_mmd,visMtp,'ERA5 Montly Prec')
# era5m_temp

In [ ]:
# */
# //--------------- Get the MEAN--------------------------------------
era5m_temp_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5m_temp.filter(ee.Filter.calendarRange(year, year, 'year')).mean()))
# .rename(['blue','green','red','nir','swir1'])))
# era5m_temp_mean

In [ ]:
# era5m_temp_mean.toBands().bandNames().getInfo()

In [ ]:
era5m_temp_mean_=era5m_temp_mean.toBands().select(era5m_temp_mean.toBands().bandNames().getInfo()
    # ['0_mean_2m_air_temperature', '1_mean_2m_air_temperature', '2_mean_2m_air_temperature', '3_mean_2m_air_temperature', '4_mean_2m_air_temperature','5_mean_2m_air_temperature', '6_mean_2m_air_temperature',
    #  '7_mean_2m_air_temperature','8_mean_2m_air_temperature','9_mean_2m_air_temperature','10_mean_2m_air_temperature'
    #  ,'11_mean_2m_air_temperature','12_mean_2m_air_temperature','13_mean_2m_air_temperature'
     ,[
 'era5m_2010_mean_temp','era5m_2011_mean_temp','era5m_2012_mean_temp','era5m_2013_mean_temp', 'era5m_2014_mean_temp',
 'era5m_2015_mean_temp', 'era5m_2016_mean_temp','era5m_2017_mean_temp', 'era5m_2018_mean_temp' ,'era5m_2019_mean_temp'
 ,'era5m_2020_mean_temp'
#  ,'era5m_2021_mean_temp','era5m_2022_mean_temp' ,'era5m_2023_mean_temp'
 ])
# era5m_temp_mean_.getInfo()

**Monthly MIn Temperature**

In [ ]:
# // Monthly MIn Temperature
era5m_min_temp = ee.ImageCollection('ECMWF/ERA5/MONTHLY').select('minimum_2m_air_temperature').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# era5m_min_temp

In [ ]:
# //--------------- Get the MEAN--------------------------------------
era5m_min_temp_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5m_min_temp.filter(ee.Filter.calendarRange(year, year, 'year')).mean().set('Year',year)))
# era5m_min_temp_mean

In [ ]:
era5m_min_temp_mean_=era5m_min_temp_mean.toBands().select(era5m_min_temp_mean.toBands().bandNames().getInfo()
    # ['0_minimum_2m_air_temperature', '1_minimum_2m_air_temperature', '2_minimum_2m_air_temperature', '3_minimum_2m_air_temperature', '4_minimum_2m_air_temperature','5_minimum_2m_air_temperature', '6_minimum_2m_air_temperature', '7_minimum_2m_air_temperature',
    #  '8_minimum_2m_air_temperature','9_minimum_2m_air_temperature','10_minimum_2m_air_temperature'
    #  ,'11_minimum_2m_air_temperature','12_minimum_2m_air_temperature','13_minimum_2m_air_temperature']
    ,[
 'era5m_2010_min_temp','era5m_2011_min_temp','era5m_2012_min_temp','era5m_2013_min_temp', 'era5m_2014_min_temp',
 'era5m_2015_min_temp','era5m_2016_min_temp','era5m_2017_min_temp', 'era5m_2018_min_temp', 'era5m_2019_min_temp','era5m_2020_min_temp'
#  ,'era5m_2021_min_temp','era5m_2022_min_temp' ,'era5m_2023_min_temp'
 ])
# era5m_min_temp_mean_.getInfo()

**Monthly Max Temperature**

In [ ]:
# // Daily Max Temperature
era5m_max_temp = ee.ImageCollection('ECMWF/ERA5/MONTHLY').select('maximum_2m_air_temperature').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# era5m_max_temp

In [ ]:
# */
# //--------------- Get the MEAN--------------------------------------
era5m_max_temp_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5m_max_temp.filter(ee.Filter.calendarRange(year, year, 'year')).mean().set('Year',year)))
# era5m_max_temp_mean

In [ ]:
era5m_max_temp_mean_=era5m_max_temp_mean.toBands().select(era5m_max_temp_mean.toBands().bandNames().getInfo()
    #   ['0_maximum_2m_air_temperature', '1_maximum_2m_air_temperature', '2_maximum_2m_air_temperature',
    #    '3_maximum_2m_air_temperature', '4_maximum_2m_air_temperature','5_maximum_2m_air_temperature',
    #    '6_maximum_2m_air_temperature', '7_maximum_2m_air_temperature',
    #  '8_maximum_2m_air_temperature','9_maximum_2m_air_temperature','10_maximum_2m_air_temperature'
    #  ,'11_maximum_2m_air_temperature','12_maximum_2m_air_temperature','13_maximum_2m_air_temperature']
       ,[
 'era5m_2010_max_temp','era5m_2011_max_temp','era5m_2012_max_temp','era5m_2013_max_temp', 'era5m_2014_max_temp',
 'era5m_2015_max_temp','era5m_2016_max_temp','era5m_2017_max_temp', 'era5m_2018_max_temp',
 'era5m_2019_max_temp','era5m_2020_max_temp'
#  ,'era5m_2021_max_temp' ,'era5m_2022_max_temp' ,'era5m_2023_max_temp'
 ])
# era5m_max_temp_mean_.getInfo()

# ERA 5 Wind

In [ ]:
# // Daily mean 10m u-component of wind
era5_u_wind_10m = ee.ImageCollection('ECMWF/ERA5/DAILY').select('u_component_of_wind_10m').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# era5_u_wind_10m

In [ ]:
# //--------------- Get the MEAN--------------------------------------
era5_u_wind_10m_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5_u_wind_10m.filter(ee.Filter.calendarRange(year, year, 'year')).mean()))
# .rename(['blue','green','red','nir','swir1'])))
# era5_u_wind_10m_mean

In [ ]:
# era5_u_wind_10m_mean.toBands().bandNames().getInfo()

In [ ]:
era5_u_wind_10m_mean_=era5_u_wind_10m_mean.toBands().select(era5_u_wind_10m_mean.toBands().bandNames().getInfo()
    # ['0_u_component_of_wind_10m', '1_u_component_of_wind_10m', '2_u_component_of_wind_10m'
    # , '3_u_component_of_wind_10m', '4_u_component_of_wind_10m',
    #  '5_u_component_of_wind_10m', '6_u_component_of_wind_10m', '7_u_component_of_wind_10m', '8_u_component_of_wind_10m'
    # '9_u_component_of_wind_10m','10_u_component_of_wind_10m','11_u_component_of_wind_10m','12_u_component_of_wind_10m','13_u_component_of_wind_10m']
    ,[
 'era5_2010_u_component_of_wind_10m','era5_2011_u_component_of_wind_10m','era5_2012_u_component_of_wind_10m','era5_2013_u_component_of_wind_10m',
 'era5_2014_u_component_of_wind_10m','era5_2015_u_component_of_wind_10m','era5_2016_u_component_of_wind_10m','era5_2017_u_component_of_wind_10m',
 'era5_2018_u_component_of_wind_10m', 'era5_2019_u_component_of_wind_10m','era5_2020_u_component_of_wind_10m'
#  ,'era5_2021_u_component_of_wind_10m' ,'era5_2022_u_component_of_wind_10m' ,'era5_2023_u_component_of_wind_10m'
 ])
# era5_u_wind_10m_mean_.getInfo()

**Monthly mean 10m u-component of wind**

In [ ]:
# // Monthly mean 10m u-component of wind
era5_u_wind_10monthly = ee.ImageCollection('ECMWF/ERA5/MONTHLY').select('u_component_of_wind_10m').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# era5_u_wind_10m

In [ ]:
# //--------------- Get the MEAN--------------------------------------
era5_u_wind_10monthly_mean = ee.ImageCollection.fromImages(years.map(lambda year: era5_u_wind_10monthly.filter(ee.Filter.calendarRange(year, year, 'year')).mean()))

# era5_u_wind_10monthly_mean

In [ ]:
# era5_u_wind_10monthly_mean.toBands().bandNames().getInfo()

In [ ]:
era5_u_wind_10monthly_mean_=era5_u_wind_10monthly_mean.toBands().select(era5_u_wind_10monthly_mean.toBands().bandNames().getInfo()
  #   ['0_u_component_of_wind_10m', '1_u_component_of_wind_10m', '2_u_component_of_wind_10m'
  #   , '3_u_component_of_wind_10m', '4_u_component_of_wind_10m',
  #    '5_u_component_of_wind_10m', '6_u_component_of_wind_10m', '7_u_component_of_wind_10m', '8_u_component_of_wind_10m'
  #  , '9_u_component_of_wind_10m','10_u_component_of_wind_10m'
    #  ,'11_u_component_of_wind_10m','12_u_component_of_wind_10m','13_u_component_of_wind_10m']
     ,[
'era5m_2010_u_component_of_wind_10m','era5m_2011_u_component_of_wind_10m','era5m_2012_u_component_of_wind_10m','era5m_2013_u_component_of_wind_10m',
 'era5m_2014_u_component_of_wind_10m','era5m_2015_u_component_of_wind_10m','era5m_2016_u_component_of_wind_10m','era5m_2017_u_component_of_wind_10m', 'era5m_2018_u_component_of_wind_10m'
 ,'era5m_2019_u_component_of_wind_10m','era5m_2020_u_component_of_wind_10m'
#  ,'era5m_2021_u_component_of_wind_10m'  ,'era5m_2022_u_component_of_wind_10m' ,'era5m_2023_u_component_of_wind_10m'
 ])
# era5_u_wind_10monthly_mean_.getInfo()

# CHIRPS Rainfall

In [ ]:
chirps_d = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')\
                    .filter(ee.Filter.date(start, end))\
                    .filter(ee.Filter.calendarRange(4, 8,'month'))
# chirps_dm = chirps_d.median().clip(AOI)
# Map.addLayer(chirps_dm,visChirps,'CHIRPS Daily Prec')
# chirps_d

In [ ]:
# */
# //--------------- Get the MEAN--------------------------------------
chirps_d_mean = ee.ImageCollection.fromImages(years.map(lambda year: chirps_d.filter(ee.Filter.calendarRange(year, year, 'year')).mean()))
# .rename(['blue','green','red','nir','swir1'])))
# chirps_d_mean

In [ ]:
# chirps_d_mean.toBands().bandNames().getInfo()

In [ ]:
chirps_d_mean_=chirps_d_mean.toBands().select(chirps_d_mean.toBands().bandNames().getInfo()
  # ['0_precipitation', '1_precipitation', '2_precipitation', '3_precipitation', '4_precipitation'
  # , '5_precipitation', '6_precipitation',
  #  '7_precipitation', '8_precipitation','9_precipitation','10_precipitation','11_precipitation','12_precipitation','13_precipitation']
 ,[
  'chirps_2010_precipitation','chirps_2011_precipitation','chirps_2012_precipitation','chirps_2013_precipitation','chirps_2014_precipitation',
  'chirps_2015_precipitation',  'chirps_2016_precipitation','chirps_2017_precipitation','chirps_2018_precipitation',
  'chirps_2019_precipitation','chirps_2020_precipitation','chirps_2021_precipitation','chirps_2022_precipitation','chirps_2023_precipitation'
  ])
# chirps_d_mean_.getInfo()

**Monthly Chirps Rainfall**

In [ ]:
chirps_m = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').select('precipitation').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# chirps_mm = chirps_d.median().clip(AOI)
# Map.addLayer(chirps_mm,visChirps,'CHIRPS Montly Prec')
# chirps_m

In [ ]:
# */
# //--------------- Get the MEAN--------------------------------------
chirps_m_mean = ee.ImageCollection.fromImages(years.map(lambda year: chirps_m.filter(ee.Filter.calendarRange(year, year, 'year')).mean()))
# .rename(['blue','green','red','nir','swir1'])))
# chirps_m_mean

In [ ]:
# chirps_m_mean.toBands().bandNames().getInfo()

In [ ]:
chirps_m_mean_=chirps_m_mean.toBands().select(chirps_m_mean.toBands().bandNames().getInfo()
  # ['0_precipitation', '1_precipitation', '2_precipitation', '3_precipitation', '4_precipitation'
  # , '5_precipitation', '6_precipitation', '7_precipitation', '8_precipitation',
  #  '9_precipitation', '10_precipitation', '11_precipitation', '12_precipitation', '13_precipitation' ]
   ,[
  'chirpsm_2010_precipitation','chirpsm_2011_precipitation','chirpsm_2012_precipitation','chirpsm_2013_precipitation','chirpsm_2014_precipitation',
  'chirpsm_2015_precipitation','chirpsm_2016_precipitation','chirpsm_2017_precipitation','chirpsm_2018_precipitation',
  'chirpsm_2019_precipitation','chirpsm_2020_precipitation','chirpsm_2021_precipitation','chirpsm_2022_precipitation','chirpsm_2023_precipitation'
  ])
# chirps_m_mean_.getInfo()

# MODIS LST

In [ ]:
start= ee.Date('2010-04-01')
end = ee.Date('2023-08-31')
years = ee.List.sequence(2010,2023)

In [ ]:
modislst_m = ee.ImageCollection('MODIS/061/MOD11A1').select('LST_Day_1km').filter(ee.Filter.date(start, end)).filter(ee.Filter.calendarRange(4, 8,'month'))
# modislst_m

In [ ]:
# */
# //--------------- Get the MEAN--------------------------------------
modislst_m_mean = ee.ImageCollection.fromImages(years.map(lambda year: modislst_m.filter(ee.Filter.calendarRange(year, year, 'year')).mean()))
# .rename(['blue','green','red','nir','swir1'])))
# modislst_m_mean

In [ ]:
# modislst_m_mean.toBands().bandNames().getInfo()

In [ ]:
modislst_m_mean_=modislst_m_mean.toBands().select(modislst_m_mean.toBands().bandNames().getInfo()
#   ['0_LST_Day_1km', '1_LST_Day_1km', '2_LST_Day_1km', '3_LST_Day_1km', '4_LST_Day_1km', '5_LST_Day_1km', '6_LST_Day_1km', '7_LST_Day_1km',
#  '8_LST_Day_1km', '9_LST_Day_1km', '10_LST_Day_1km', '11_LST_Day_1km', '12_LST_Day_1km', '13_LST_Day_1km' ]
  ,[
  'modis_2010_lst','modis_2011_lst','modis_2012_lst','modis_2013_lst','modis_2014_lst',
  'modis_2015_lst','modis_2016_lst','modis_2017_lst','modis_2018_lst',
  'modis_2019_lst','modis_2020_lst','modis_2021_lst','modis_2022_lst','modis_2023_lst'])
# modislst_m_mean_.getInfo()

# MERGE IMGCOL

In [ ]:
imageList = [era5_tp_mean_,
             era5_monthly_mean_,
             era5_mean_temp_mean_,
             era5_min_temp_mean_,
             era5_max_temp_mean_,
             era5m_temp_mean_,
             era5m_min_temp_mean_,
             era5m_max_temp_mean_,
             era5_u_wind_10m_mean_,
             era5_u_wind_10monthly_mean_,
             chirps_d_mean_,chirps_m_mean_,
             modislst_m_mean_,
             l8_2010,l8_2011,l8_2012,l8_2013,l8_2014,l8_2015,l8_2016,l8_2017,l8_2018,l8_2019,l8_2020,l8_2021,l8_2022]

# imageList = [chirps_d_mean_,chirps_m_mean_]

In [ ]:
# // Merge the list of images into an image collection
imgcol = ee.ImageCollection.fromImages(imageList)


In [ ]:
imgcol_ = imgcol.toBands()
# imgcol_.getInfo()

In [ ]:
imgcol_repr = imgcol_.reproject(crs='EPSG:4326', scale=30)

In [ ]:
asset_id = ee.FeatureCollection("projects/ee-maizeyield/assets/LR_18/rf_12_20")


In [ ]:
# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)


In [ ]:
# feature_names = [ 'nsi','gli', 'sipi','era5_wind', 'era5m_prec','era5_mean_temp', 'era5_min_temp', 'era5_max_temp',
#         'era5m_mean_temp','chirpsm_prec']

feature_names = ['era5d_prec','era5_min_temp','gpp','npp', 'wdvi','chirpsd_prec','gli','gvi','arvi2']

In [ ]:
# 'COUNTY', 'Year', 'ndvi', 'savi', 'evi', 'gndvi', 'ndmi', 'nsi',
#        'msavi', 'gci', 'gli', 'sipi', 'sr', 'osavi', 'gci2', 'arvi2', 'mtvi2',
#        'rdvi', 'gvi', 'ndii', 'ndwi', 'grvi', 'lai', 'tcari', 'mcari', 'wdvi',
#        'pssra', 'dvi', 'era5d_prec', 'era5_wind', 'era5m_wind', 'era5m_prec',
#        'era5_mean_temp', 'era5_min_temp', 'era5_max_temp', 'era5m_mean_temp',
#        'era5m_min_temp', 'era5m_max_temp', 'chirpsd_prec', 'chirpsm_prec',
#        'gpp','npp', 'Yield2', 'mod_lst'

# 2010

In [ ]:
year = '2010'
start = ee.Date('2010-04-01')
end = ee.Date('2010-08-31')


In [ ]:
gpp_col = ee.ImageCollection("MODIS/061/MOD17A3HGF")#.select('Gpp')
npp_col = ee.ImageCollection("MODIS/061/MOD17A3HGF")#.select('Gpp')


In [ ]:
era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[0]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[0]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[0]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_2010 = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp_2010 = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2010.select('wdvi')
gli = l8_2010.select('gli')
gvi =l8_2010.select('gvi')
arvi2 =l8_2010.select('arvi2')


In [ ]:
img_2010 = era5d_prec.addBands(era5_min_temp)\
.addBands(chirpsd_prec)\
.addBands(npp_2010)\
.addBands(gpp_2010)\
.addBands(wdvi)\
.addBands(gli)\
.addBands(gvi)\
.addBands(arvi2)

# .addBands(era5m_min_temp)\# .addBands(era5m_wind)\
# .addBands(era5m_max_temp)\
# .addBands(chirpsd_prec)\# .addBands(gpp)\
# .addBands(npp)\
# .addBands(mod_lst)

In [ ]:
rsn2 = gpp_2010.select('gpp').projection()
# print(rsn2.nominalScale())


ee.Number({
  "functionInvocationValue": {
    "functionName": "Projection.nominalScale",
    "arguments": {
      "proj": {
        "functionInvocationValue": {
          "functionName": "Image.projection",
          "arguments": {
            "image": {
              "functionInvocationValue": {
                "functionName": "Image.select",
                "arguments": {
                  "bandSelectors": {
                    "constantValue": [
                      "gpp"
                    ]
                  },
                  "input": {
                    "functionInvocationValue": {
                      "functionName": "Image.rename",
                      "arguments": {
                        "input": {
                          "functionInvocationValue": {
                            "functionName": "Image.select",
                            "arguments": {
                              "bandSelectors": {
                                "functionInvocationValue": {
   

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=500)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_2010_r = gpp_2010.reproject(rsn2)
npp_2010_r = npp_2010.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

In [ ]:
# era5d_prec_r.resample('bilinear').reproject(crs=crs, scale=scale)

In [ ]:
img_2010_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_2010_r)\
.addBands(gpp_2010_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)

In [ ]:
img_2010_r.bandNames().getInfo()

['era5d_prec',
 'era5_min_temp',
 'chirpsd_prec',
 'npp',
 'gpp',
 'wdvi',
 'gli',
 'gvi',
 'arvi2']

In [ ]:
# classify the image again but with the classifier from the persisted trees
# classified = img_2010.select(feature_names).classify(another_classifier)
classified = img_2010_r.select(feature_names).classify(another_classifier)

In [ ]:
# Reduce Regions to Counties
class_2010 = classified.reduceRegions(**{
    'collection':AOI,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})


In [ ]:
Map.addLayer(newzones,{},'Newzones')
Map

Map(bottom=4525.0, center=[-5.659718554577286, 48.27392578125001], controls=(WidgetControl(options=['position'…

In [ ]:
# Reduce Regions to Counties
class_2010_n = img_2010_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})



In [ ]:
# class_2010_n = class_2010_n.map(lambda f: ee.Algorithms.If(f.geometry().coordinates().size(), f, None))

In [ ]:
# class_2010_n.aggregate_count('constant').getInfo()

0

In [ ]:
# Reduce Regions to Counties
class_2010_n = img_2010_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2010_n, folder='ee_demos', description='class_2010_n', fileFormat='SHP')
task.start()

In [ ]:
# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2010',
#   'assetId': 'users/Prod/Outputs/Class2010_newZ',
#   'scale': 463.31271652791656,
#   'region': county.geometry()
#  })
# task.start()

In [ ]:
# task = ee.batch.Export.image.toDrive(**{
#   'image': classified,
#   'description': 'Class2010',
#   'folder':'ee_demos',
#   'scale': scale,
#   'region': AOI,
#   'crs':'EPSG:4326'
#  })

# # task.start()

In [ ]:
task.status()

{'state': 'COMPLETED',
 'description': 'class_2010_n',
 'priority': 100,
 'creation_timestamp_ms': 1740645432524,
 'update_timestamp_ms': 1740645503018,
 'start_timestamp_ms': 1740645440032,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://drive.google.com/#folders/1PnnfRcWoguMmhUR6tgBDpXFIi7ueBPJR'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 75.51627349853516,
 'id': 'OXNH7TD3P42OYPNUPY7FYYOG',
 'name': 'projects/ee-maizeyield/operations/OXNH7TD3P42OYPNUPY7FYYOG'}

# 2011

In [ ]:
year ='2011'
start = ee.Date('2011-04-01')
end = ee.Date('2011-08-31')
scale = 463.31271652791656

In [ ]:
era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[1]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[1]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[1]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[1]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[1]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[1]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[1]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[1]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[1]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[1]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[1]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[1]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[1]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2011.select('wdvi')
gli = l8_2011.select('gli')
gvi =l8_2011.select('gvi')
arvi2 =l8_2011.select('arvi2')



In [ ]:
img_2011 = era5d_prec.addBands(era5_min_temp)\
.addBands(chirpsd_prec)\
.addBands(npp)\
.addBands(gpp)\
.addBands(wdvi)\
.addBands(gli)\
.addBands(gvi)\
.addBands(arvi2)

# .addBands(era5m_min_temp)\# .addBands(era5m_wind)\
# .addBands(era5m_max_temp)\
# .addBands(chirpsd_prec)\# .addBands(gpp)\
# .addBands(npp)\
# .addBands(mod_lst)

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=500)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

In [ ]:
img_2011_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)

In [ ]:
# classify the image again but with the classifier from the persisted trees
classified = img_2011_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2011 = classified.reduceRegions(**{
    'collection':AOI,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})
# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2011',
#   'assetId': 'users/Prod/Outputs/Out5k/Class2011',
#   'scale': scale,
#   'region': county.geometry()
# })

# task.start()

In [ ]:
# Reduce Regions to Counties
class_2011_n = img_2011_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2011_n, folder='ee_demos', description='class_2011_n', fileFormat='SHP')
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2011',
  'folder':'ee_demos',
  'scale': scale,
  'region': AOI,
  'crs':'EPSG:4326'
 })

# task.start()

In [ ]:
 task.status()

{'state': 'RUNNING',
 'description': 'class_2011_n',
 'priority': 100,
 'creation_timestamp_ms': 1740645684201,
 'update_timestamp_ms': 1740645724397,
 'start_timestamp_ms': 1740645694219,
 'task_type': 'EXPORT_FEATURES',
 'attempt': 1,
 'batch_eecu_usage_seconds': 1.774157762,
 'id': '2SYBQTISRU5HJ3EAPSV7XTIY',
 'name': 'projects/ee-maizeyield/operations/2SYBQTISRU5HJ3EAPSV7XTIY'}


# 2012

In [ ]:
year ='2012'
start = ee.Date('2012-04-01')
end = ee.Date('2012-08-31')
scale = 463.31271652791656

In [ ]:
era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[2]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[2]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[2]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[2]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[2]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[2]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[2]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[2]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[2]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[2]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[2]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[2]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[2]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2012.select('wdvi')
gli = l8_2012.select('gli')
gvi =l8_2012.select('gvi')
arvi2 =l8_2012.select('arvi2')


In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=500)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

img_2012_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)
classified = img_2012_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2012 = classified.reduceRegions(**{
    'collection':AOI,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})

task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2012',
  'folder':'ee_demos',
  'scale': scale,
  'region': AOI,
  'crs':'EPSG:4326'
 })

# task.start()

In [ ]:
# Reduce Regions to Counties
class_2012_n = img_2012_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2010_n, folder='ee_demos', description='class_2012_n', fileFormat='SHP')
task.start()

In [ ]:
task.status()

{'state': <State.UNSUBMITTED: 'UNSUBMITTED'>}

# 2013

In [ ]:
year ='2013'
start = ee.Date('2013-04-01')
end = ee.Date('2013-08-31')
scale = 463.31271652791656

In [ ]:

era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[3]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[3]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[3]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[3]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[3]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[3]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[3]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[3]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[3]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[3]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[3]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[3]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[3]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2013.select('wdvi')
gli = l8_2013.select('gli')
gvi =l8_2013.select('gvi')
arvi2 =l8_2013.select('arvi2')

# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2013',
#   'assetId': 'users/Prod/Outputs/Out5k/Class2013',
#   'scale': scale,
#   'region': county.geometry()
# })
# task.start()

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=500)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

img_2013_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)
classified = img_2013_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2013 = classified.reduceRegions(**{
    'collection':county,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})

task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2013',
  'folder':'ee_demos',
  'scale': scale,
  'region': county.geometry(),
  'crs':'EPSG:4326'
 })

# task.start()

In [ ]:
# Reduce Regions to Counties
class_2013_n = img_2013_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2013_n, folder='ee_demos', description='class_2013_n', fileFormat='SHP')
task.start()

In [ ]:
task.status()

{'state': 'RUNNING',
 'description': 'class_2013_n',
 'priority': 100,
 'creation_timestamp_ms': 1740645885498,
 'update_timestamp_ms': 1740645896412,
 'start_timestamp_ms': 1740645896244,
 'task_type': 'EXPORT_FEATURES',
 'attempt': 1,
 'id': 'WWDUDY6MIRGPIVMHKJWIIND5',
 'name': 'projects/ee-maizeyield/operations/WWDUDY6MIRGPIVMHKJWIIND5'}

# 2014

In [ ]:
year ='2014'
start = ee.Date('2014-04-01')
end = ee.Date('2014-08-31')
scale = 463.31271652791656

In [ ]:

era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[4]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[4]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[4]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[4]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[4]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[4]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[4]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[4]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[4]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[4]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[4]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[4]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[4]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2014.select('wdvi')
gli = l8_2014.select('gli')
gvi =l8_2014.select('gvi')
arvi2 =l8_2014.select('arvi2')
# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2014',
#   'assetId': 'users/Prod/Outputs/Out5k/Class2014',
#   'scale': scale,
#   'region': county.geometry()
# })
# task.start()

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=500)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

img_2014_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)

classified = img_2014_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2014 = classified.reduceRegions(**{
    'collection':county,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})

task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2014',
  'folder':'ee_demos',
  'scale': scale,
  'region': county.geometry(),
  'crs':'EPSG:4326'
 })

# task.start()

In [ ]:
# Reduce Regions to Counties
class_2014_n = img_2014_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2014_n, folder='ee_demos', description='class_2014_n', fileFormat='SHP')
task.start()

In [ ]:
task.status()

{'state': 'READY',
 'description': 'class_2014_n',
 'priority': 100,
 'creation_timestamp_ms': 1740645921248,
 'update_timestamp_ms': 1740645927075,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': 'JFWVESW7O6HN3APY4TAT6U7U',
 'name': 'projects/ee-maizeyield/operations/JFWVESW7O6HN3APY4TAT6U7U'}

# 2015

In [ ]:
year ='2015'
start = ee.Date('2015-04-01')
end = ee.Date('2015-08-31')
scale = 463.31271652791656

In [ ]:

era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[5]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[5]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[5]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[5]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[5]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[5]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[5]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[5]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[5]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[5]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[5]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[5]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[5]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2015.select('wdvi')
gli = l8_2015.select('gli')
gvi =l8_2015.select('gvi')
arvi2 =l8_2015.select('arvi2')

# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2015',
#   'assetId': 'users/Prod/Outputs/Out5k/Class2015',
#   'scale': scale,
#   'region': county.geometry()
# })
# task.start()

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=500)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

img_2015_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)

classified = img_2015_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2015 = classified.reduceRegions(**{
    'collection':county,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})

task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2015',
  'folder':'ee_demos',
  'scale': scale,
  'region': county.geometry(),
  'crs':'EPSG:4326'
 })

# task.start()

In [ ]:
# Reduce Regions to Counties
class_2015_n = img_2015_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2015_n, folder='ee_demos', description='class_2015_n', fileFormat='SHP')
task.start()

In [ ]:
task.status()

{'state': 'READY',
 'description': 'class_2015_n',
 'priority': 100,
 'creation_timestamp_ms': 1740645962712,
 'update_timestamp_ms': 1740645962712,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': '44EWL4KBOOTKWMCUIBA4UMIP',
 'name': 'projects/ee-maizeyield/operations/44EWL4KBOOTKWMCUIBA4UMIP'}


# 2016

In [ ]:
year ='2016'
start = ee.Date('2016-04-01')
end = ee.Date('2016-08-31')
scale = 463.31271652791656

In [ ]:

era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[6]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[6]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[6]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[6]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[6]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[6]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[6]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[6]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[6]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[6]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[6]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[6]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[6]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2016.select('wdvi')
gli = l8_2016.select('gli')
gvi =l8_2016.select('gvi')
arvi2 =l8_2016.select('arvi2')

# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2016',
#   'assetId': 'users/Prod/Outputs/Out5k/Class2016',
#   'scale': scale,
#   'region': county.geometry()
# })
# task.start()

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=500)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

img_2016_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)

classified = img_2016_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2016 = classified.reduceRegions(**{
    'collection':county,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})

task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2016',
  'folder':'ee_demos',
  'scale': scale,
  'region': county.geometry(),
  'crs':'EPSG:4326'
 })

# task.start()

In [ ]:
# Reduce Regions to Counties
class_2016_n = img_2016_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2016_n, folder='ee_demos', description='class_2016_n', fileFormat='SHP')
task.start()

In [ ]:
task.status()

{'state': 'READY',
 'description': 'class_2016_n',
 'priority': 100,
 'creation_timestamp_ms': 1740646093885,
 'update_timestamp_ms': 1740646093885,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': 'HCHWTIWK2OYLPSNJZKEV3NB4',
 'name': 'projects/ee-maizeyield/operations/HCHWTIWK2OYLPSNJZKEV3NB4'}

# 2017

In [ ]:
year ='2017'
start = ee.Date('2017-04-01')
end = ee.Date('2017-08-31')
scale = 463.31271652791656

In [ ]:

era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[7]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[7]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[7]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[7]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[7]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[7]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[7]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[7]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[7]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[7]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[7]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[7]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[7]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2017.select('wdvi')
gli = l8_2017.select('gli')
gvi =l8_2017.select('gvi')
arvi2 =l8_2017.select('arvi2')

# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2017',
#   'assetId': 'users/Prod/Outputs/Out5k/Class2017',
#   'scale': scale,
#   'region': county.geometry()
# })
# task.start()

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=500)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

img_2017_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)

classified = img_2017_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2017 = classified.reduceRegions(**{
    'collection':county,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})

task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2017',
  'folder':'ee_demos',
  'scale': scale,
  'region': county.geometry(),
  'crs':'EPSG:4326'
 })

# task.start()

In [ ]:
# Reduce Regions to Counties
class_2017_n = img_2017_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2017_n, folder='ee_demos', description='class_2017_n', fileFormat='SHP')
task.start()

In [ ]:
task.status()

{'state': 'READY',
 'description': 'class_2017_n',
 'priority': 100,
 'creation_timestamp_ms': 1740646111696,
 'update_timestamp_ms': 1740646111696,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': 'PGWY6HPJNIDWS4E7QASLC7CU',
 'name': 'projects/ee-maizeyield/operations/PGWY6HPJNIDWS4E7QASLC7CU'}

# 2018

In [ ]:
year ='2018'
start = ee.Date('2018-04-01')
end = ee.Date('2018-08-31')
scale = 463.31271652791656

In [ ]:

era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[8]).rename('era5d_prec')
era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[8]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[8]).rename('era5m_wind')
era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[8]).rename('era5m_prec')
era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[8]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[8]).rename('era5_min_temp')
era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[8]).rename('era5_max_temp')
era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[8]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[8]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[8]).rename('era5m_max_temp')
# chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[8]).rename('chirpsd_prec')
chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[8]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[8]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2018.select('wdvi')
gli = l8_2018.select('gli')
gvi =l8_2018.select('gvi')
arvi2 =l8_2018.select('arvi2')

# img_2018 = era5d_prec.addBands(era5_wind)\
# .addBands(era5m_prec)\
# .addBands(era5_mean_temp)\
# .addBands(era5_min_temp)\
# .addBands(era5_max_temp)\
# .addBands(era5m_mean_temp)\
# .addBands(chirpsm_prec)\
# .addBands(nsi)\
# .addBands(gli)\
# .addBands(sipi)

# # .addBands(era5m_min_temp)\# .addBands(era5m_wind)\
# # .addBands(era5m_max_temp)\
# # .addBands(chirpsd_prec)\# .addBands(gpp)\
# # .addBands(npp)\
# # .addBands(mod_lst)
# # classify the image again but with the classifier from the persisted trees
# classified = img_2018.select(feature_names).classify(another_classifier)
# # Reduce Regions to Counties
# class_2018 = classified.reduceRegions(**{
#     'collection':county,
#     'reducer': ee.Reducer.mean(),
#     'scale': scale
# })
# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2018',
#   'assetId': 'users/Prod/Outputs/Out5k/Class2018',
#   'scale': scale,
#   'region': county.geometry()
# })
# task.start()

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=scale)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

img_2018_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)

classified = img_2018_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2018 = classified.reduceRegions(**{
    'collection':AOI,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})

task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2018',
  'folder':'ee_demos',
  'scale': scale,
  'region': AOI,
  'crs':'EPSG:4326'
 })



In [ ]:
# Reduce Regions to Counties
class_2018_n = img_2018_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2018_n, folder='ee_demos', description='class_2018_n', fileFormat='SHP')
task.start()

In [ ]:
# task.start()
task.status()

{'state': 'READY',
 'description': 'class_2018_n',
 'priority': 100,
 'creation_timestamp_ms': 1740646151039,
 'update_timestamp_ms': 1740646151039,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': 'A7GSQXZYZNWTHT2FUZ44XJ2X',
 'name': 'projects/ee-maizeyield/operations/A7GSQXZYZNWTHT2FUZ44XJ2X'}

# 2019

In [ ]:
year ='2019'
start = ee.Date('2019-04-01')
end = ee.Date('2019-08-31')
scale = 463.31271652791656

In [ ]:

era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[9]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[9]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[9]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[9]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[9]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[9]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[9]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[9]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[9]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[9]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[9]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[9]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[9]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2019.select('wdvi')
gli = l8_2019.select('gli')
gvi =l8_2019.select('gvi')
arvi2 =l8_2019.select('arvi2')


# img_2019 = era5d_prec.addBands(era5_wind)\
# .addBands(era5m_prec)\
# .addBands(era5_mean_temp)\
# .addBands(era5_min_temp)\
# .addBands(era5_max_temp)\
# .addBands(era5m_mean_temp)\
# .addBands(chirpsm_prec)\
# .addBands(nsi)\
# .addBands(gli)\
# .addBands(sipi)

# # .addBands(era5m_min_temp)\# .addBands(era5m_wind)\
# # .addBands(era5m_max_temp)\
# # .addBands(chirpsd_prec)\# .addBands(gpp)\
# # .addBands(npp)\
# # .addBands(mod_lst)
# # classify the image again but with the classifier from the persisted trees
# classified = img_2019.select(feature_names).classify(another_classifier)
# # Reduce Regions to Counties
# class_2019 = classified.reduceRegions(**{
#     'collection':county,
#     'reducer': ee.Reducer.mean(),
#     'scale': scale
# })
# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2019',
#   'assetId': 'users/Prod/Outputs/Out5k/Class2019',
#   'scale': scale,
#   'region': county.geometry()
# })
# task.start()

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=scale)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

img_2019_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)

classified = img_2019_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2019 = classified.reduceRegions(**{
    'collection':AOI,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})

task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2019',
  'folder':'ee_demos',
  'scale': scale,
  'region': AOI,
  'crs':'EPSG:4326'
 })

# task.start()

In [ ]:
# Reduce Regions to Counties
class_2019_n = img_2019_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2019_n, folder='ee_demos', description='class_2019_n', fileFormat='SHP')
task.start()

In [ ]:
# Reduce Regions to Counties
class_2019_n = img_2019_r.reduceRegions(**{
    'collection':newzones,
    'reducer': ee.Reducer.median(),
    'scale': scale,
     'crs':'EPSG:4326'
})

task = ee.batch.Export.table.toDrive(collection=class_2019_n, folder='ee_demos', description='class_2019_n', fileFormat='SHP')
task.start()


In [ ]:
# task.start()
task.status()

{'state': 'COMPLETED',
 'description': 'class_2019_n',
 'priority': 100,
 'creation_timestamp_ms': 1740651263576,
 'update_timestamp_ms': 1740651409794,
 'start_timestamp_ms': 1740651273436,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://drive.google.com/#folders/1PnnfRcWoguMmhUR6tgBDpXFIi7ueBPJR'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 514.9331665039062,
 'id': 'EBTDWBETGFOF5CEYGDFR5B3Y',
 'name': 'projects/ee-maizeyield/operations/EBTDWBETGFOF5CEYGDFR5B3Y'}

# 2020

In [ ]:
year ='2020'
start = ee.Date('2020-04-01')
end = ee.Date('2020-08-31')
scale = 463.31271652791656

In [ ]:

era5d_prec = era5_tp_mean_.select(era5_tp_mean_.bandNames().getInfo()[10]).rename('era5d_prec')
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[10]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[10]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[10]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[10]).rename('era5_mean_temp')
era5_min_temp =  era5_min_temp_mean_.select(era5_min_temp_mean_.bandNames().getInfo()[10]).rename('era5_min_temp')
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[10]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[10]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[10]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[10]).rename('era5m_max_temp')
chirpsd_prec =chirps_d_mean_.select(chirps_d_mean_.bandNames().getInfo()[10]).rename('chirpsd_prec')
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[10]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[10]).rename('mod_lst')
gpp = gpp_prep(gpp_col,AOI,year).toBands().select(gpp_prep(gpp_col,AOI,year).toBands().bandNames()).rename('gpp')
npp = npp_prep(npp_col,AOI,year).toBands().select(npp_prep(npp_col,AOI,year).toBands().bandNames()).rename('npp')
wdvi = l8_2020.select('wdvi')
gli = l8_2020.select('gli')
gvi =l8_2020.select('gvi')
arvi2 =l8_2020.select('arvi2')


# img_2020 = era5d_prec.addBands(era5_wind)\
# .addBands(era5m_prec)\
# .addBands(era5_mean_temp)\
# .addBands(era5_min_temp)\
# .addBands(era5_max_temp)\
# .addBands(era5m_mean_temp)\
# .addBands(chirpsm_prec)\
# .addBands(nsi)\
# .addBands(gli)\
# .addBands(sipi)

# # .addBands(era5m_min_temp)\# .addBands(era5m_wind)\
# # .addBands(era5m_max_temp)\
# # .addBands(chirpsd_prec)\# .addBands(gpp)\
# # .addBands(npp)\
# # .addBands(mod_lst)
# # classify the image again but with the classifier from the persisted trees
# classified = img_2020.select(feature_names).classify(another_classifier)
# # Reduce Regions to Counties
# class_2020 = classified.reduceRegions(**{
#     'collection':county,
#     'reducer': ee.Reducer.mean(),
#     'scale': scale
# })
# task = ee.batch.Export.image.toAsset(**{
#   'image': classified,
#   'description': 'Class2020',
#   'assetId': 'users/Prod/Outputs/Out5k/Class2020',
#   'scale': scale,
#   'region': county.geometry()
# })
# task.start()

In [ ]:
# img_2010_repr = img_2010.reproject(crs='EPSG:4326', scale=500)
era5d_prec_r = era5d_prec.resample().reproject(rsn2)
# era5_wind = era5_u_wind_10m_mean_.select(era5_u_wind_10m_mean_.bandNames().getInfo()[0]).rename('era5_wind')
# era5m_wind = era5_u_wind_10monthly_mean_.select(era5_u_wind_10monthly_mean_.bandNames().getInfo()[0]).rename('era5m_wind')
# era5m_prec = era5_monthly_mean_.select(era5_monthly_mean_.bandNames().getInfo()[0]).rename('era5m_prec')
# era5_mean_temp = era5_mean_temp_mean_.select(era5_mean_temp_mean_.bandNames().getInfo()[0]).rename('era5_mean_temp')
era5_min_temp_r =  era5_min_temp.resample().reproject(rsn2)
# era5_max_temp = era5_max_temp_mean_.select(era5_max_temp_mean_.bandNames().getInfo()[0]).rename('era5_max_temp')
# era5m_mean_temp = era5m_temp_mean_.select(era5m_temp_mean_.bandNames().getInfo()[0]).rename('era5m_mean_temp')
# era5m_min_temp = era5m_min_temp_mean_.select(era5m_min_temp_mean_.bandNames().getInfo()[0]).rename('era5m_min_temp')
# era5m_max_temp = era5m_max_temp_mean_.select(era5m_max_temp_mean_.bandNames().getInfo()[0]).rename('era5m_max_temp')
chirpsd_prec_r =chirpsd_prec.resample().reproject(rsn2)
# chirpsm_prec =chirps_m_mean_.select(chirps_m_mean_.bandNames().getInfo()[0]).rename('chirpsm_prec')
# mod_lst = modislst_m_mean_.select(modislst_m_mean_.bandNames().getInfo()[0]).rename('mod_lst')
gpp_r = gpp.reproject(rsn2)
npp_r = npp.reproject(rsn2)
wdvi_r = wdvi.reproject(rsn2)
gli_r = gli.reproject(rsn2)
gvi_r = gvi.reproject(rsn2)
arvi2_r =arvi2.reproject(rsn2)

img_2020_r = era5d_prec_r.addBands(era5_min_temp_r)\
.addBands(chirpsd_prec_r)\
.addBands(npp_r)\
.addBands(gpp_r)\
.addBands(wdvi_r)\
.addBands(gli_r)\
.addBands(gvi_r)\
.addBands(arvi2_r)

classified = img_2020_r.select(feature_names).classify(another_classifier)
# Reduce Regions to Counties
class_2020 = classified.reduceRegions(**{
    'collection':AOI,
    'reducer': ee.Reducer.mean(),
    'scale': scale
})

task = ee.batch.Export.image.toDrive(**{
  'image': classified,
  'description': 'Class2020',
  'folder':'ee_demos',
  'scale': scale,
  'region': AOI,
  'crs':'EPSG:4326'
 })

# task.start()

In [ ]:
# task.start()
task.status()

{'state': <State.UNSUBMITTED: 'UNSUBMITTED'>}

In [ ]:
# class_2020 = classified.reduceRegions(**{
#     'collection':county,
#     'reducer': ee.Reducer.mean(),
#     'scale': scale
# })

In [ ]:
# geemap.ee_export_vector(class_2020, 'class_2020.csv')


# 2021

# 2022

# 2023

In [ ]:
# Map.addLayer(classified)

# EXPORT CSV

In [ ]:
# geemap.ee_export_vector(test_reg, 'climatemod.csv')


In [ ]:
# Define the visualization parameters.
# vizParams = {
#   'bands': ['B5', 'B4', 'B3'],
#   'min': 0,
#   'max': 0.5,
#   'gamma': [0.95, 1.1, 1]
# }


In [ ]:
# Map.addLayer(test_reg,{},'Climate Variables')
# Map